In [1]:
import pandas as pd

import datajoint as dj
from commons import mice
import static_nda as nda
from pipeline import experiment

Connecting pfahey@at-database.ad.bcm.edu:3306
Loading local settings from pipeline_config.json


In [2]:
# scan_keys = nda.Scan # scans considered for release, including duplicates
scan_keys = nda.ScanInclude # scans included in release

In [3]:
animal_keys = dj.U('animal_id') & scan_keys
surgery_rest = {'surgery_type':'Cranial Window and Headbar',
                'surgery_outcoome':'Survival'}

animal_dicts = []
for animal_key in animal_keys:
    animal_dict = {**animal_key}
    lines,genotypes = (mice.Genotypes & animal_key).fetch('line','genotype',order_by='line')
    animal_dict['genotype_clean'] = ', '.join([' '.join([l,g]) for l,g in zip(lines,genotypes) 
                                               if g != 'negative'])
    animal_dict['genotype_full'] = ', '.join([' '.join([l,g]) for l,g in zip(lines,genotypes)])
    dob_rel = dj.U('animal_id','sex','dob') * (mice.Mice & animal_key)
    
    first_rel = dob_rel.aggr(experiment.Scan,
                           first_scan_ts = 'CAST(min(scan_ts) AS DATE)',
                           keep_all_rows=True)
    inc_rel = dob_rel.aggr(experiment.Scan & scan_keys,
                           first_inc_scan_ts = 'CAST(min(scan_ts) AS DATE)',
                           keep_all_rows=True)
    surg_rel = (experiment.Surgery & animal_key & surgery_rest).proj(surg_ts = 'date')
    dob_rel = dob_rel * surg_rel * first_rel * inc_rel 
    
    dob_rel = dob_rel.proj('sex','dob','surg_ts','first_scan_ts','first_inc_scan_ts',
                           age_at_surgery = 'DATEDIFF(surg_ts,dob)',
                           age_at_first_scan = 'DATEDIFF(first_scan_ts,dob)',
                           age_at_first_inc_scan = 'DATEDIFF(first_inc_scan_ts,dob)')
    
    animal_dict = {**animal_dict, **dob_rel.fetch(as_dict=True)[0]}
    animal_dicts.append(animal_dict)
animal_df = pd.DataFrame(animal_dicts)

display(animal_df)

print('\n\n\n animal sex distribution')
display(animal_df[['animal_id','sex']].groupby('sex').count())

print('\n\n\n animal genotype distribution')
display(animal_df[['animal_id','genotype_clean']].groupby('genotype_clean').count())
display(animal_df[['animal_id','genotype_full']].groupby('genotype_full').count())




,animal_id,genotype_clean,genotype_full,dob,sex,surgery_id,surg_ts,first_scan_ts,first_inc_scan_ts,age_at_surgery,age_at_first_scan,age_at_first_inc_scan
0,21067,"Ai162_GCamp6 heterozygous, SLC-Cre heterozygous","Ai162_GCamp6 heterozygous, SLC-Cre heterozygous",2018-12-21,F,1,2019-03-15,2019-03-18,2019-04-03,84,87,103
1,22846,"Ai162_GCamp6 heterozygous, SLC-Cre heterozygous","Ai162_GCamp6 heterozygous, SLC-Cre heterozygous",2019-08-01,M,1,2019-11-04,2019-11-04,2019-12-04,95,95,125
2,23343,"Ai162_GCamp6 heterozygous, SLC-Cre heterozygous","Ai162_GCamp6 heterozygous, SLC-Cre heterozygous",2019-10-29,M,1,2019-12-25,2020-01-07,2020-01-15,57,70,78
3,23656,"Ai162_GCamp6 heterozygous, SLC-Cre heterozygous","Ai162_GCamp6 heterozygous, Dlx5-CreER negative...",2019-12-17,F,1,2020-03-19,2020-03-19,2020-04-30,93,93,135
4,23964,"Ai162_GCamp6 heterozygous, SLC-Cre heterozygous","Ai162_GCamp6 heterozygous, Dlx5-CreER negative...",2020-02-05,F,1,2020-06-10,2020-06-10,2020-06-29,126,126,145
5,26644,"Ai162_GCamp6 heterozygous, SLC-Cre heterozygous","Ai162_GCamp6 heterozygous, Dlx5-CreER negative...",2021-09-23,M,1,2021-11-23,2021-11-30,2021-12-13,61,68,81
6,26645,"Ai162_GCamp6 heterozygous, SLC-Cre heterozygous","Ai162_GCamp6 heterozygous, Dlx5-CreER negative...",2021-09-23,M,1,2021-11-23,2021-12-09,2021-12-14,61,77,82





 animal sex distribution


,animal_id
sex,
F,3
M,4





 animal genotype distribution


,animal_id
genotype_clean,
"Ai162_GCamp6 heterozygous, SLC-Cre heterozygous",7


,animal_id
genotype_full,
"Ai162_GCamp6 heterozygous, Dlx5-CreER negative, SLC-Cre heterozygous",4
"Ai162_GCamp6 heterozygous, SLC-Cre heterozygous",3
